# NAME SURNAME

## Packages

In [3]:
import networkx as nx # For graphs
import pickle # For data parsing
import math
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split, Dataset
from networkx.algorithms.approximation import greedy_tsp # For approx TSP

## Helper functions

In [4]:

def tour_length(G, tour):
    """
    Compute the length of a tour. A tour is a list having elments 0 and -1 equal
    """
    n = len(tour) - 1
    assert tour[0] == tour[-1], "Not valid tour"
    estimated = 0
    for i in range(n):
        estimated += G[tour[i]][tour[i + 1]]['weight']
    return estimated

def greedy_algorithm(G):
    """
    Run the value of the greedy approximation algorithm on graph G
    """
    return tour_length(G, greedy_tsp(G, weight='weight'))

def random_tour(G, seed = 42):
    """
    Return the value of a random tour
    """
    np.random.seed(seed)
    n = G.number_of_nodes()
    tour = [0]
    for i in range(1, n):
        next_node = np.random.choice([j for j in range(n) if j not in tour])
        tour.append(next_node)
    tour.append(0)
    return tour_length(G, tour)
    

def transformer_tsp(G, model, DEVICE = 'cpu'):
    """
    Evaluate your (trained) model on G
    TODO: If you used some masks, add them when needed. 
    """
    # Set the model in evaluation mode
    model.eval()

    # Note: number of edges is constant ed equal to n(n-1)/2
    n = G.number_of_nodes()
    E = G.number_of_edges()

    
    # Get node coordinates
    attr = nx.get_node_attributes(G, 'pos')
    x = []
    for i in range(n):
        x.append(torch.tensor(attr[i], dtype=torch.float32))

    # From list of tensors to tensor
    x = torch.stack(x)    

    tour = [0]
    y = torch.tensor(tour, dtype=torch.long)
    x = torch.stack(x)
    x = x.to(DEVICE).unsqueeze(0)
    y = y.to(DEVICE).unsqueeze(0)
    
    out = model(x, y)
    
    while len(tour) < n:
        _, idx = torch.topk(out, n, dim=2)
        for i in range(n):
            if idx[0, 0, i] not in tour:
                tour.append(idx[0, 0, i])
                break
        y = torch.tensor(tour)
        y = y.to(DEVICE).unsqueeze(0)
        out = model(x, y)
    
    tour = [int(i) for i in tour] + [0]
    return tour_length(G, tour)



def gap(G, model = None, model_GA = None, random_seed = 42, device = 'cpu'):
    """
    Compute the gap between the optimal solution on graph G and all the analyzed methods
    """
        
    # Optimal value (hard-coded in the graph)
    TSP = sum([G[i][j]['weight']*G[i][j]['tour'] for (i, j) in G.edges()]) # Optimal

    # Gaps dictionary
    gaps = {'greedy' : 0, 'random' : 0, 'transformer_tsp': 0, 'transformer_tsp_acc_grad': 0}
    gaps['greedy'] = 100* (greedy_algorithm(G) -  TSP) / TSP
    gaps['random'] = 100 * (random_tour(G, random_seed) - TSP) / TSP
    if model is not None:
        gaps['transformer_tsp'] = 100 * (transformer_tsp(G, model, DEVICE=device) - TSP) / TSP
    else:
        gaps['transformer_tsp'] = float('inf')
        
    if model_GA is not None:
        gaps['transformer_tsp_acc_grad'] = 100 * (transformer_tsp(G, model_GA, DEVICE=device) - TSP) / TSP
    else:
        gaps['transformer_tsp_acc_grad'] = float('inf')
    return gaps    
    

## Dataset & Dataloader

### Dataset Point 1

In [ ]:
# Load the dummy dataset
with open('/kaggle/input/tsp-dataset/dummy_20_DLL_ass4.pkl', 'rb') as file:
   dummy = pickle.load(file)

# Print type of overall dataset and a single item
print(f"Type of dummy dataset is: {type(dummy)}")
single_item = dummy[0]
print(f"\nType of a single item is: {type(single_item)}")

# Print types of elements in the tuple
print(f"\nTypes of elements in the tuple:")
print(f"First element type: {type(single_item[0])}") # Graph
print(f"Second element type: {type(single_item[1])}") # List

print("\Single Item structure:")
print(single_item)

### Dataset Point 2

In [ ]:
# Describe the edge attributes tour and weight, as well as the node attribute pos.


graph = single_item[0] 
tour = single_item[1]  # in this case tour as a list

# Describe node attribute: pos
node_positions = nx.get_node_attributes(graph, 'pos')
print("\nNode positions (pos):")
for node, pos in node_positions.items():
    print(f"Node {node}: Position {pos}")

# Describe edge attribute: weight
print("\nEdge weights:")
for u, v in graph.edges():
    weight = graph[u][v].get('weight', None) 
    print(f"Edge ({u}, {v}): Weight {weight}")

# Describe edge attribute: tour
print("\nTour:")
print(f"Tour as per solution: {tour}")





### Dataset Point 4

In [ ]:
# Dataset Class
import torch
from torch.utils.data import Dataset

class TSPDataset(Dataset):
    def __init__(self, data):
        """
        Args:
            data (list): A list where each element is a tuple (graph, tour).
                         - graph: Graph object containing node positions as 'pos' attribute.
                         - tour: List of node indices representing the tour.
        """
        self.data = data

    def __len__(self):
        """
        Return the number of samples in the dataset.
        """
        return len(self.data)

    def __getitem__(self, idx):
        """
        Args:
            idx (int): Index of the data sample to retrieve.
        
        Returns:
            X (torch.Tensor): A tensor of shape (20, 2) representing node coordinates.
            y (torch.Tensor): A tensor of shape (n+1,) representing the tour starting and ending at 0.
        """
        graph, tour = self.data[idx]

        # Extract node positions from the graph
        pos = nx.get_node_attributes(graph, 'pos')  # Assumes graph stores positions in 'pos'
        X = torch.tensor([pos[node] for node in sorted(pos.keys())], dtype=torch.float32)

        # Ensure the tour starts and ends at node 0
        if tour[0] != 0:
            raise ValueError("Tour must start at node 0.")
        if tour[-1] != 0:
            tour = tour + [0]  # Append 0 to ensure the tour ends at the start node

        y = torch.tensor(tour, dtype=torch.long)

        return X, y


### Dataset Point 5

In [ ]:
# Create Dataset objects for training, validation, and testing, along with their respective Dataloader

tspdataset = TSPDataset(dummy)
train_dataset, validation_dataset, test_dataset = random_split(tspdataset, [0.8, 0.1, 0.1])

batch_size = 32

trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True)
testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)





## Model

In [ ]:

class TSPTransformer(nn.Module):
    def __init__(self, d_model=128, nhead=8, num_encoder_layers=6, 
                 num_decoder_layers=6, dim_feedforward=512, dropout=0.1):
        super(TSPTransformer, self).__init__()
        
        # Input embeddings for node coordinates
        self.input_embedding = nn.Linear(2, d_model)  # 2D coordinates to d_model
        
        # Output embeddings for positions
        self.output_embedding = nn.Embedding(20, d_model)  # Assuming max 20 nodes
        
        # Positional encoding for decoder
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        
        # Transformer base
        self.transformer = nn.Transformer(
            d_model=d_model,
            nhead=nhead,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True
        )
        
        # Final feed-forward network
        self.ffn = nn.Sequential(
            nn.Linear(d_model, dim_feedforward),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(dim_feedforward, 20)  # Output size = number of nodes
        )
    
    def create_mask(self, size):
        # Create mask to prevent attending to future positions
        mask = (torch.triu(torch.ones(size, size)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf'))
        mask = mask.masked_fill(mask == 1, float(0.0))
        return mask
        
    def forward(self, src, tgt):
        # src shape: [batch_size, num_nodes, 2]
        # tgt shape: [batch_size, seq_len]
        
        # Create masks
        tgt_mask = self.create_mask(tgt.size(1)).to(src.device)
        src_padding_mask = None  # All nodes are valid
        tgt_padding_mask = None  # All positions are valid
        
        # Embed inputs
        src = self.input_embedding(src)  # [batch_size, num_nodes, d_model]
        
        # Embed outputs and add positional encoding
        tgt = self.output_embedding(tgt)  # [batch_size, seq_len, d_model]
        tgt = self.pos_encoder(tgt)
        
        # Transformer forward pass
        output = self.transformer(
            src, tgt,
            tgt_mask=tgt_mask,
            src_mask=None,
            memory_mask=None,
            src_key_padding_mask=src_padding_mask,
            tgt_key_padding_mask=tgt_padding_mask,
            memory_key_padding_mask=None
        )
        
        # Final feed-forward network
        output = self.ffn(output)
        
        return output

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=100):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def generate_square_subsequent_mask(sz):
    ## Decoder mask
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

# Just to have an idea of what it looks like:
print(generate_square_subsequent_mask(5))

In [ ]:
# Inizializzazione del modello
model = TSPTransformer(
    d_model=128,            # Dimensione dello spazio di embedding
    nhead=8,                # Numero di teste di attention
    num_encoder_layers=6,   # Numero di layer dell'encoder
    num_decoder_layers=6,   # Numero di layer del decoder
    dim_feedforward=512,    # Dimensione del layer feed-forward
    dropout=0.1            # Dropout per regolarizzazione
)

# Sposta il modello su GPU se disponibile
model = model.to(DEVICE)

print(model)

## Training

### Training WITHOUT gradient accumulation

### Training WITH gradient accumulation

## Testing